這裡開始已經假設你已經看過前面的所有基礎文件說明，因此多數註解會拿掉以維護版面乾淨

AlexNet是劃時代的一個重要作品，2012年橫空出世，讓整個電腦視覺的世界有了改變。後續的作品不少都受它影響。引入dropout、可視化第一層的filter，雙GPU、ReLU做為activation function、很深的神經網路...等。都是裡面的經典論述。不過裡面提到的Local Response Normalization在後續被證明沒有效果?因此後續就乏人問津了。

在下已有翻譯AlexNet論文，也可以參閱[相關文件](https://hackmd.io/@shaoeChen/SyjI6W2zB/https%3A%2F%2Fhackmd.io%2F%40shaoeChen%2FSJK_0YJmI)

首先載入相關需求套件

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.1.0'

指定硬體資源

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

資料集的部份是使用ImageNet訓練，不過這部份在下就只提供[資料集連結](http://www.image-net.org/)，不然硬train一發怕時間太久。

從論文中我們知道：
* Alexnet的input dimension為224x224x3，從256x256的照片中隨機取得224x224的照片
    * 這邊不知道是寫錯還是怎麼樣，input dimension為227x227的時候，其後續的dimension才會與論文上的架構圖相同
* 過程中使用不少的資料增強的方式，這部份可以透過keras舊有的generator來達成，並不需要再自己實作一個，不過這取決你的建模方式
* 他使用雙GPU訓練，不過現在隨便一張1080、2080系列的記憶體都不少，應該可以一張達成
* 第一層用了非常大的filter size 11x11來進行卷積，並且利用比較大的stride來降低後面的資料維度

利用標準的keras Sequential來建置模型

In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(227, 227, 3)),   
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1000, activation='softmax')    
])

確認模型

In [18]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 13, 13, 256)      

編譯模型

In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

事實上，雖然AlexNet在目前來看好像是一個過氣的架構，但要注意到的是，不是新的架構就一定滿足你的資料集，這部份通常都是調整、調整、再調整才有辦法確定怎麼樣的架構適合怎麼樣的資料集，不要落入"潮"的思維陷井中。